In [1]:
 !pip install scanpy
 %matplotlib inline
!python -m pip uninstall matplotlib -Y
!pip install matplotlib==3.1.3
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached matplotlib-3.6.2-cp38-cp38-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (9.4 MB)
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.1.3
    Uninstalling matplotlib-3.1.3:
      Successfully uninstalled matplotlib-3.1.3



Usage:   
  /usr/bin/python3 -m pip uninstall [options] <package> ...
  /usr/bin/python3 -m pip uninstall [options] -r <requirements file> ...

no such option: -Y
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached matplotlib-3.1.3-cp38-cp38-manylinux1_x86_64.whl (13.1 MB)
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.6.2
    Uninstalling matplotlib-3.6.2:
      Successfully uninstalled matplotlib-3.6.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scanpy 1.9.1 requires matplotlib>=3.4, but you have matplotlib 3.1.3 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import torch
from torch import nn, optim
import torch.nn.functional as F

from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets, transforms
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import scanpy as sc
import plotly.express as px
import plotly.io as pio
import sklearn.preprocessing
import sklearn.model_selection





torch.autograd.set_detect_anomaly(True)

In [3]:
import platform

def get_device_and_gmount():
    # Get the operating system and version
    os = platform.system()
    version = platform.release()

    # Get the machine's architecture
    arch = platform.machine()

    # Set the default renderer based on the operating system
    if os == 'Darwin':
        pio.renderers.default = 'notebook'
        print("Using Apple MPS on Macbook Pro")
    
    elif os == 'Linux':
        pio.renderers.default = 'colab'
        print("Using Colab on Linux")

    # Set the device based on the machine's architecture
    if arch == 'x86_64':
        device = torch.device('mps') if os == 'Darwin' else torch.device('cuda')
        gmount = True if os == 'Linux' else False
    else:
        device = torch.device('cpu')
        gmount = False

    print("Using device:", device)
    
    return device, gmount


In [4]:
device, gmount = get_device_and_gmount()


Using Apple MPS on Macbook Pro
Using device: mps


In [5]:
if gmount:
    from google.colab import drive
    drive.mount('/content/drive')
    scdata = sc.read_h5ad("/content/drive/MyDrive/scintegration/GEX.h5ad")
else:
    scdata = sc.read_h5ad("/Users/eamonmcandrew/Desktop/Single_cell_integration/Data/Multi-ome/GEX.h5ad")

In [6]:
def stratified_split(data, test_size, random_state, split_criteria):
    """
    Splits the data into train and test sets stratified by the batch column
    """
    train = []
    test = []
    for batch in data.obs[split_criteria].unique():
        batch_data = data[data.obs[split_criteria] == batch]
        batch_train, batch_test = sklearn.model_selection.train_test_split(batch_data, test_size=test_size, random_state=random_state)
        batch_train, batch_test = list(batch_train.obs.index), list(batch_test.obs.index)
        train.extend(batch_train)
        test.extend(batch_test)
        
    return train, test


In [7]:
if gmount == True:
    from google.colab import drive
    drive.mount('/content/drive')
    scdata = sc.read_h5ad("/content/drive/MyDrive/scintegration/GEX.h5ad")

In [8]:
import wandb
wandb.login()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: eamomc. Use `wandb login --relogin` to force relogin


True

In [9]:
class GEX_Dataset(torch.utils.data.Dataset):
    def __init__(self, data, scaler=None, cat_var=None, label_encoder=None):
        self.data = data
        self.values = np.asarray(data.X.todense())
        self.cat_var = cat_var

        label_encoder_functions = {
            "numeric": lambda: torch.tensor(sklearn.preprocessing.LabelEncoder().fit_transform(self.data.obs[self.cat_var]), dtype=torch.long),
            "range_map": lambda: sklearn.preprocessing.LabelEncoder().fit_transform(self.data.obs[self.cat_var]).reshape(-1, 1),
            "one_hot": lambda: sklearn.preprocessing.OneHotEncoder().fit_transform(sklearn.preprocessing.LabelEncoder().fit_transform(self.data.obs[self.cat_var]).reshape(-1, 1)).toarray()
        }

        if label_encoder in label_encoder_functions:
            cat_var_data = label_encoder_functions[label_encoder]()
            if label_encoder == "range_map":
                cat_var_data = torch.tensor(sklearn.preprocessing.MinMaxScaler().fit_transform(cat_var_data), dtype=torch.float32)
            elif label_encoder == "one_hot":
                cat_var_data = torch.tensor(cat_var_data, dtype=torch.float32)
        else:
            cat_var_data = None
        self.cat_var_data = cat_var_data

        scaler_functions = {
            "Standard": lambda: sklearn.preprocessing.StandardScaler().fit_transform(self.values),
            "MinMax": lambda: sklearn.preprocessing.MinMaxScaler().fit_transform(self.values)
        }

        if scaler in scaler_functions:
            self.scaled_values = torch.tensor(scaler_functions[scaler](), dtype=torch.float32)
        else:
            self.scaled_values = torch.tensor(self.values, dtype=torch.float32)

    @property
    def n_features(self):
        return self.values.shape[1]

    @property
    def n_catagories(self):
        return self.cat_var_data.shape[1] if self.cat_var_data is not None else 0

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.scaled_values[idx], self.cat_var_data[idx]


In [10]:
def stratified_split(data, test_size, random_state, split_criteria):
    """
    Splits the data into train and test sets stratified by the batch column
    """
    train = []
    test = []
    for batch in data.obs[split_criteria].unique():
        batch_data = data[data.obs[split_criteria] == batch]
        batch_train, batch_test = sklearn.model_selection.train_test_split(batch_data, test_size=test_size, random_state=random_state)
        batch_train, batch_test = list(batch_train.obs.index), list(batch_test.obs.index)
        train.extend(batch_train)
        test.extend(batch_test)
        
    return train, test


In [11]:
class Generator(nn.Module):
    def __init__(self, input_size, dropout, hidden_sizes, output_size, use_batch_norm):
        super(Generator, self).__init__()

        self.use_batch_norm = use_batch_norm

        # Create a list of Linear layers with the specified hidden sizes
        self.hidden_layers = [nn.Linear(input_size, hidden_size) for hidden_size in hidden_sizes]
        # Create a BatchNorm1d layer for each hidden layer if use_batch_norm is True
        self.batch_norm_layers = [nn.BatchNorm1d(hidden_size) for hidden_size in hidden_sizes] if use_batch_norm else []
        # Create a Dropout layer for each hidden layer
        self.dropout_layers = [nn.Dropout(dropout) for _ in hidden_sizes]
        # Create a Linear layer for the output
        self.output_layer = nn.Linear(hidden_sizes[-1], output_size)

    def forward(self, x):
        # Loop through the hidden layers
        for hidden_layer, batch_norm_layer, dropout_layer in zip(self.hidden_layers, self.batch_norm_layers, self.dropout_layers):
            # Apply the hidden layer, batch norm layer (if use_batch_norm is True), and dropout layer
            x = hidden_layer(x)
            if self.use_batch_norm:
                x = batch_norm_layer(x)
            x = F.relu(x)
            x = dropout_layer(x)
        # Apply the output layer and return the output
        x = self.output_layer(x)
        return x


In [12]:
Generator(10, 0.5, [20, 20], 10, True)

Generator(
  (output_layer): Linear(in_features=20, out_features=10, bias=True)
)

In [11]:
class Discriminator(nn.Module):
    def __init__(self, input_size, dropout, hidden_sizes, output_size, use_batch_norm):
        super(Discriminator, self).__init__()

        self.use_batch_norm = use_batch_norm

        # Create a list of Linear layers with the specified hidden sizes
        self.hidden_layers = [nn.Linear(input_size, hidden_size) for hidden_size in hidden_sizes]
        # Create a BatchNorm1d layer for each hidden layer if use_batch_norm is True
        self.batch_norm_layers = [nn.BatchNorm1d(hidden_size) for hidden_size in hidden_sizes] if use_batch_norm else []
        # Create a Dropout layer for each hidden layer
        self.dropout_layers = [nn.Dropout(dropout) for _ in hidden_sizes]
        # Create a Linear layer for the output
        self.output_layer = nn.Linear(hidden_sizes[-1], output_size)

    def forward(self, x):
        # Loop through the hidden layers
        for hidden_layer, batch_norm_layer, dropout_layer in zip(self.hidden_layers, self.batch_norm_layers, self.dropout_layers):
            # Apply the hidden layer, batch norm layer (if use_batch_norm is True), and dropout layer
            x = hidden_layer(x)
            if self.use_batch_norm:
                x = batch_norm_layer(x)
            x = F.relu(x)
            x = dropout_layer(x)
        # Apply the output layer and return the output
        x = self.output_layer(x)
        return torch.sigmoid(x)


In [12]:
# Define the GAN model
class GAN(nn.Module):
    def __init__(self, generator, discriminator):
        super().__init__()
        self.generator = generator
        self.discriminator = discriminator

    def forward(self, x):
        return self.discriminator(self.generator(x))

In [13]:
def generate_fake_data_and_combine(original_data, generator, z_dim, batch_size):
    generator.eval()
    noise = torch.randn(batch_size, z_dim)
    fake_data = generator(noise)
    fake_adata = sc.AnnData(fake_data.detach().numpy())

    # add an index column to the fake data, starting at the number of real cells
    fake_adata.var.index = scdata.var.index
    adata = sc.AnnData.concatenate(original_data, fake_adata)
    # add a "group" column to the observations DataFrame,
    # with the value "real" for the real cells and "fake" for the fake cells
    adata.obs['group'] = ['real' if i < len(scdata.obs) else 'fake' for i in range(len(adata.obs))]

    # compute the UMAP of the combined data
    sc.pp.neighbors(adata, n_neighbors=10)
    sc.tl.umap(adata)

    return adata

In [ ]:
def plot_umap_by_group(adata):
    plot = sc.pl.umap(adata, color='group', legend_loc='on data', show=False)
    return plot 

In [14]:
def train_gan_one_epoch(epoch, GEX_dataloader_train, gan, optimizer, criterion):
    gan.train()
    for iteration, (data, _) in enumerate(GEX_dataloader_train):
        data = data.to(device)
        # Generate fake data
        noise = torch.randn(data.shape[0], z_dim)
        noise = noise.to(device)
        fake_data = gan.generator(noise)

        # Train the discriminator
        optimizer.zero_grad()
        pred_real = gan.discriminator(data)
        pred_fake = gan.discriminator(fake_data.detach())
        loss_real = criterion(pred_real, torch.ones_like(pred_real))
        loss_fake = criterion(pred_fake, torch.zeros_like(pred_fake))
        loss_discriminator = (loss_real + loss_fake) / 2
        loss_discriminator.backward()

        # Train the generator
        pred_fake = gan.discriminator(fake_data)
        loss_generator = criterion(pred_fake, torch.ones_like(pred_fake))
        loss_generator.backward()
        optimizer.step()

    return loss_discriminator.item(), loss_generator.item()


In [ ]:
config = wandb.config = {
    "lr": 0.001,
    "batch_size": 64,
    "epochs": 60,
    "random_seed": 42,
    "dropout_G": 0,
    "dropout_D": 0.2,
    "split_criteria": "cell_type",
    "eval_size_percentage": 0.1,
    "hidden_sizes_G": [32, 64, 128],
    "hidden_sizes_D": [32, 64, 128],
    "use_batch_norm_G": True,
    "use_batch_norm_D": True,
    "z_dim": 100,
}


In [16]:
def train_func(config):
    
    if config is not None:
        config = wandb.config
        run = wandb.init(project="Single Cell Omics integration", entity="scintegration", config=config)
    else:
      run = wandb.init(project="Single Cell Omics integration", entity="scintegration")
    

    # Load the learning rate, batch size, epochs, random seed, dropout, and hidden size from the wandb configuration

    lr = wandb.config.lr
    batch_size = wandb.config.batch_size
    epochs = wandb.config.epochs
    random_seed = wandb.config.random_seed
    dropout_G = wandb.config.dropout_G
    dropout_D = wandb.config.dropout_D
    split_criteria = wandb.config.split_criteria
    eval_size_percentage = wandb.config.eval_size_percentage
    hidden_sizes_G = wandb.config.hidden_sizes_G
    hidden_sizes_D = wandb.config.hidden_sizes_D
    use_batch_norm_G = wandb.config.use_batch_norm_G
    use_batch_norm_D = wandb.config.use_batch_norm_D
    z_dim = wandb.config.z_dim


    train, test = stratified_split(scdata, 0.2, wandb.config.random_seed, split_criteria=split_criteria)
    train_data = scdata[train]
    test_data = scdata[test]


    GEX_Dataset_train = GEX_Dataset(train_data, scaler="Standard", cat_var="batch", label_encoder="one_hot")
    GEX_dataloader_train = torch.utils.data.DataLoader(GEX_Dataset_train, batch_size = 128, shuffle = True)

    GEX_Dataset_test = GEX_Dataset(test_data, scaler="Standard", cat_var="batch", label_encoder="one_hot")
    GEX_dataloader_test = torch.utils.data.DataLoader(GEX_Dataset_test, batch_size = 128, shuffle = True)

    output_size = GEX_Dataset.n_features
    target_size = GEX_Dataset.n_features

    # Create instance of the generator and discriminator networks
    generator = Generator(input_size=z_dim, hidden_sizes=hidden_sizes_G, output_size=target_size, use_batch_norm = use_batch_norm_G, dropout=dropout_G)

    discriminator = Discriminator(input_size=target_size, hidden_sizes=hidden_sizes_D, output_size=1, use_batch_norm = use_batch_norm_D, dropout=dropout_D)

    gan = GAN(generator, discriminator)
    gan = gan.to(device)

    loss_fn = nn.BCELoss()
    optimizer = torch.optim.Adam(gan.parameters(), lr=lr)
    
    test_UMAP = sc.pl.umap(test_data, color="cell_type", show=False)
    wandb.log({"Test UMAP": wandb.Image(test_UMAP)})
    # Create instance of the GAN model
    gan = GAN(generator, discriminator)
    gan.train()
    for epoch in range(1,epochs):
      D_loss_train, G_loss_train = train_gan_one_epoch(epoch, GEX_dataloader_train, gan, optimizer, loss_fn)
      wandb.log({"D_loss_train": D_loss_train, "G_loss_train": G_loss_train})
      if epoch % 3 == 0:
        umap_data = generate_fake_data_and_combine(test_data, generator, z_dim, batch_size)
        umap = plot_umap_by_group(umap_data)
        wandb.log({"UMAP": umap})

      




In [19]:
wandb.